In [ ]:
# Import python packages
import streamlit as st

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Assuming the dataset 
# https://huggingface.co/datasets/gradio/NYC-Airbnb-Open-Data
# has been downloaded to DATABASE.SCHEMA

DATABASE = 'snowpark_playground'
SCHEMA = 'hug_datasets'
TABLE='nyc_airbnb_open_data'

session.use_database(DATABASE)
session.use_schema(SCHEMA)

In [ ]:
st.image('Airbnb Notebook.jpeg', width=800)

In [ ]:
select current_database(), current_schema();

In [ ]:
select * from nyc_airbnb_open_data sample (1000 rows);
-- show tables;

In [ ]:
st.map(property_data.to_df().collect(), size='number_of_reviews')

In [ ]:
select count(*) as property_count
from snowpark_playground.hug_datasets.nyc_airbnb_open_data;

In [ ]:
create schema if not exists airbnb;
CREATE
OR REPLACE VIEW airbnb.NYC_AIRBNB_OPEN_DATA_WITH_PRICE_INDICATOR AS WITH avg_price AS (
  SELECT
    AVG("price") AS avg_price
  FROM
    SNOWPARK_PLAYGROUND.HUG_DATASETS.NYC_AIRBNB_OPEN_DATA
),
avg_reviews AS (
  SELECT
    AVG("number_of_reviews") AS avg_reviews
  FROM
    SNOWPARK_PLAYGROUND.HUG_DATASETS.NYC_AIRBNB_OPEN_DATA
),
avg_availability AS (
  SELECT
    AVG("availability_365") AS avg_availability
  FROM
    SNOWPARK_PLAYGROUND.HUG_DATASETS.NYC_AIRBNB_OPEN_DATA
)
SELECT
  "price" as METRIC, "neighbourhood", "neighbourhood_group", "room_type",
  CASE
    WHEN "price" > (
      SELECT
        avg_price
      FROM
        avg_price
    ) THEN TRUE
    ELSE FALSE
  END AS is_price_above_average,
  CASE
    WHEN "number_of_reviews" > (
      SELECT
        avg_reviews
      FROM
        avg_reviews
    ) THEN 'high'
    WHEN "number_of_reviews" = (
      SELECT
        avg_reviews
      FROM
        avg_reviews
    ) THEN 'medium'
    ELSE 'low'
  END AS review_volume,
  CASE
    WHEN "availability_365" > (
      SELECT
        avg_availability * 1.1
      FROM
        avg_availability
    ) THEN 'high'
    WHEN "availability_365" >= (
      SELECT
        avg_availability * 0.9
      FROM
        avg_availability
    ) THEN 'medium'
    ELSE 'low'
  END AS availability
FROM
  SNOWPARK_PLAYGROUND.HUG_DATASETS.NYC_AIRBNB_OPEN_DATA;

In [ ]:
use schema snowpark_playground.airbnb;
select * from NYC_AIRBNB_OPEN_DATA_WITH_PRICE_INDICATOR
limit 10;

In [ ]:
-- Create Instance of Top Insights 
CREATE SNOWFLAKE.ML.TOP_INSIGHTS IF NOT EXISTS airbnb_price_insights();

-- Function to get key drivers 
CALL airbnb_price_insights!GET_DRIVERS(
    INPUT_DATA => TABLE(NYC_AIRBNB_OPEN_DATA_WITH_PRICE_INDICATOR),
    LABEL_COLNAME => 'is_price_above_average',
    METRIC_COLNAME => 'metric');

-- Create a Table based on Top Insights Results.
CREATE OR REPLACE TABLE airbnb_pricing_results as select * from table(result_scan(-1));

In [ ]:
SELECT

contributor,
contribution, 
relative_contribution, 
growth_rate

FROM airbnb_pricing_results

-- WHERE contributor NOT LIKE '%not%'

In [ ]:
SELECT

contributor,

SNOWFLAKE.CORTEX.COMPLETE('llama3.1-405b', CONCAT ('You are a business analyst that is going to analyze the top contributors to Aibnb prices. The Contributor field represents the relevant segments.',contributor, 'The RELATIVE_CONTRIBUTION field measures how imporant the contributors are',relative_contribution,'Return a concise summary how the contributor impacts Airbnb prices. Do not say "Based on the data, here is a concise summary". Do not return information not captured in the input data.')) as Top_Insights_Row_Summary, 
contribution, 
relative_contribution, 
growth_rate

FROM airbnb_pricing_results

-- WHERE contributor NOT LIKE '%not%'

GROUP BY ALL 


In [ ]:
with top_insights_input as (

SELECT

contributor,

SNOWFLAKE.CORTEX.COMPLETE('llama3.1-405b', CONCAT ('You are a business analyst that is going to analyze the top contributors to Aibnb prices. The Contributor field represents the relevant segments.',contributor, 'The RELATIVE_CONTRIBUTION field measures how imporant the contributors are',relative_contribution,'Return a concise summary how the contributor impacts Airbnb prices. Do not say "Based on the data, here is a concise summary". Do not return information not captured in the input data.')) as Top_Insights_Row_Summary, 
contribution, 
relative_contribution, 
growth_rate

FROM airbnb_pricing_results

-- WHERE contributor NOT LIKE '%not%'

GROUP BY ALL 

)

SELECT 

SNOWFLAKE.CORTEX.COMPLETE('llama3.1-405b', CONCAT ('You are a real estate analyst writing a short blog post about Airbnb prices in New York Ciy based on this summary:',LEFT(LISTAGG(Top_Insights_Row_Summary),100000),'Return a well written summary no longer than 500 words with the key takeaways for Airbnb Hosts that will be published for readers who are thinking about listing their apartments in New York City')) as Top_Insights_Row_Summary


FROM top_insights_input 


In [ ]:
result=blog_post.to_df()
st.markdown(blog_post.to_df().collect()[0][0])